In [1]:
!pip3 install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.1 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch 
from indicnlp.tokenize import indic_tokenize
import torch.nn.functional as F

ModuleNotFoundError: No module named 'indicnlp'

In [3]:
data =pd.read_csv("/kaggle/input/sample-dataset/Sentence pairs in English-Hindi - 2025-02-11.tsv",sep="\t",header=None,names=["SrcSentId","SrcSent","DataSentId","DstSent"])

In [4]:
data

,SrcSentId,SrcSent,DataSentId,DstSent
0,1282,Muiriel is 20 now.,485968,म्यूरियल अब बीस साल की हो गई है।
1,1282,Muiriel is 20 now.,2060319,म्यूरियल अब बीस साल की है।
2,1294,Education in this world disappoints me.,485564,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,1302,That won't happen.,2060320,वैसा नहीं होगा।
4,1308,I miss you.,2060321,मुझें तुम्हारी याद आ रही है।
...,...,...,...,...
13177,12886228,Do you have some curry and some rice?,12886225,क्या आपके पास सब्ज़ी-चावल है?
13178,12886229,Do you have curry and some rice?,12886225,क्या आपके पास सब्ज़ी-चावल है?
13179,12886230,Do you have any curry with rice?,12886225,क्या आपके पास सब्ज़ी-चावल है?
13180,12886231,Do you have curry with rice?,12886225,क्या आपके पास सब्ज़ी-चावल है?


In [5]:
data.head()

,SrcSentId,SrcSent,DataSentId,DstSent
0,1282,Muiriel is 20 now.,485968,म्यूरियल अब बीस साल की हो गई है।
1,1282,Muiriel is 20 now.,2060319,म्यूरियल अब बीस साल की है।
2,1294,Education in this world disappoints me.,485564,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,1302,That won't happen.,2060320,वैसा नहीं होगा।
4,1308,I miss you.,2060321,मुझें तुम्हारी याद आ रही है।


In [6]:
data.shape

(13182, 4)

In [7]:
data.drop(labels=[data.columns[0],data.columns[2]],axis=1,inplace=True)

In [8]:
data.head()

,SrcSent,DstSent
0,Muiriel is 20 now.,म्यूरियल अब बीस साल की हो गई है।
1,Muiriel is 20 now.,म्यूरियल अब बीस साल की है।
2,Education in this world disappoints me.,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,That won't happen.,वैसा नहीं होगा।
4,I miss you.,मुझें तुम्हारी याद आ रही है।


In [9]:
src_sent_tokenizer=AutoTokenizer.from_pretrained("google-T5/T5-base")

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
data["SrcSent"]= data["SrcSent"].apply(lambda x: src_sent_tokenizer.tokenize(x))

In [11]:
data.head()

,SrcSent,DstSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की हो गई है।
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की है।
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...",मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,"[▁That, ▁won, ', t, ▁happen, .]",वैसा नहीं होगा।
4,"[▁I, ▁miss, ▁you, .]",मुझें तुम्हारी याद आ रही है।


In [12]:
data["DstSent"]= data["DstSent"].apply(lambda x: indic_tokenize.trivial_tokenize(x,lang="hi"))

In [13]:
data.head()

,SrcSent,DstSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[▁That, ▁won, ', t, ▁happen, .]","[वैसा, नहीं, होगा, ।]"
4,"[▁I, ▁miss, ▁you, .]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [14]:
data["SrcSent"]= data["SrcSent"].apply(src_sent_tokenizer.convert_tokens_to_ids)

In [15]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[466, 751, 31, 17, 1837, 5]","[वैसा, नहीं, होगा, ।]"
4,"[27, 3041, 25, 5]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [16]:
Vs=src_sent_tokenizer.get_vocab()

In [17]:
len(Vs)

32100

In [18]:
hindi_vocab=set()
for tokenized_hindi_sent in data["DstSent"]:
    hindi_vocab.update(tokenized_hindi_sent)

In [19]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[466, 751, 31, 17, 1837, 5]","[वैसा, नहीं, होगा, ।]"
4,"[27, 3041, 25, 5]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [20]:
Vd = dict()
for idx,token in enumerate(hindi_vocab):
    Vd[token]=idx+3
Vd["<PAD>"]=0
Vd["<SOS>"]=1
Vd["<EOS>"]=2

In [21]:
len(Vd)

7072

In [22]:
print(type(data["DstSent"].iloc[0]))


<class 'list'>


In [23]:
def convert_hindi_token_to_ids(hindi_sent):
    return [Vd[token] for token in hindi_sent]

In [24]:
data["DstSent"]=data["DstSent"].apply(lambda x:convert_hindi_token_to_ids(x))

In [25]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[4711, 1098, 5710, 2502, 5554, 2298, 20, 888, ..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[4711, 1098, 5710, 2502, 5554, 888, 3547]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[741, 2827, 2727, 755, 6201, 3564, 5340, 6012,..."
3,"[466, 751, 31, 17, 1837, 5]","[2579, 6101, 1607, 3547]"
4,"[27, 3041, 25, 5]","[6706, 4774, 5484, 3797, 4598, 888, 3547]"


In [26]:
def insert_sos_token_id(hindi_token_sent_ids):
    return [1]+hindi_token_sent_ids

In [27]:
data["DstSentInput"]=data["DstSent"].apply(lambda x:insert_sos_token_id(x))
data.head()

,SrcSent,DstSent,DstSentInput
0,"[4159, 23, 14018, 19, 460, 230, 5]","[4711, 1098, 5710, 2502, 5554, 2298, 20, 888, ...","[1, 4711, 1098, 5710, 2502, 5554, 2298, 20, 88..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[4711, 1098, 5710, 2502, 5554, 888, 3547]","[1, 4711, 1098, 5710, 2502, 5554, 888, 3547]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[741, 2827, 2727, 755, 6201, 3564, 5340, 6012,...","[1, 741, 2827, 2727, 755, 6201, 3564, 5340, 60..."
3,"[466, 751, 31, 17, 1837, 5]","[2579, 6101, 1607, 3547]","[1, 2579, 6101, 1607, 3547]"
4,"[27, 3041, 25, 5]","[6706, 4774, 5484, 3797, 4598, 888, 3547]","[1, 6706, 4774, 5484, 3797, 4598, 888, 3547]"


In [28]:
def insert_eos_token_id(hindi_token_sent_ids):
    return hindi_token_sent_ids+[2]

In [29]:
data["DstSentLabel"]=data["DstSent"].apply(lambda x:insert_eos_token_id(x))
data.head()

,SrcSent,DstSent,DstSentInput,DstSentLabel
0,"[4159, 23, 14018, 19, 460, 230, 5]","[4711, 1098, 5710, 2502, 5554, 2298, 20, 888, ...","[1, 4711, 1098, 5710, 2502, 5554, 2298, 20, 88...","[4711, 1098, 5710, 2502, 5554, 2298, 20, 888, ..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[4711, 1098, 5710, 2502, 5554, 888, 3547]","[1, 4711, 1098, 5710, 2502, 5554, 888, 3547]","[4711, 1098, 5710, 2502, 5554, 888, 3547, 2]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[741, 2827, 2727, 755, 6201, 3564, 5340, 6012,...","[1, 741, 2827, 2727, 755, 6201, 3564, 5340, 60...","[741, 2827, 2727, 755, 6201, 3564, 5340, 6012,..."
3,"[466, 751, 31, 17, 1837, 5]","[2579, 6101, 1607, 3547]","[1, 2579, 6101, 1607, 3547]","[2579, 6101, 1607, 3547, 2]"
4,"[27, 3041, 25, 5]","[6706, 4774, 5484, 3797, 4598, 888, 3547]","[1, 6706, 4774, 5484, 3797, 4598, 888, 3547]","[6706, 4774, 5484, 3797, 4598, 888, 3547, 2]"


In [30]:
data.drop(labels=[data.columns[1]],axis=1,inplace=True)

In [31]:
data.head()

,SrcSent,DstSentInput,DstSentLabel
0,"[4159, 23, 14018, 19, 460, 230, 5]","[1, 4711, 1098, 5710, 2502, 5554, 2298, 20, 88...","[4711, 1098, 5710, 2502, 5554, 2298, 20, 888, ..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[1, 4711, 1098, 5710, 2502, 5554, 888, 3547]","[4711, 1098, 5710, 2502, 5554, 888, 3547, 2]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[1, 741, 2827, 2727, 755, 6201, 3564, 5340, 60...","[741, 2827, 2727, 755, 6201, 3564, 5340, 6012,..."
3,"[466, 751, 31, 17, 1837, 5]","[1, 2579, 6101, 1607, 3547]","[2579, 6101, 1607, 3547, 2]"
4,"[27, 3041, 25, 5]","[1, 6706, 4774, 5484, 3797, 4598, 888, 3547]","[6706, 4774, 5484, 3797, 4598, 888, 3547, 2]"


In [32]:
X=list(data["SrcSent"])
y_input=list(data["DstSentInput"])
y_label=list(data["DstSentLabel"])

In [33]:
X_tensor=[torch.tensor(tokenized_eng_sent_ids) for tokenized_eng_sent_ids in X]

In [34]:
Y_input_tensor= [torch.tensor(tokenized_hindi_sent_ids) for tokenized_hindi_sent_ids in y_input]

In [35]:
Y_label_tensor=[torch.tensor(tokenized_hindi_sent_ids) for tokenized_hindi_sent_ids in y_label]

In [36]:
X_padded= torch.nn.utils.rnn.pad_sequence(X_tensor,batch_first=True)


In [37]:
Y_padded_input= torch.nn.utils.rnn.pad_sequence(Y_input_tensor,batch_first=True)
Y_padded_label= torch.nn.utils.rnn.pad_sequence(Y_label_tensor,batch_first=True)

In [38]:
Ns=X_padded.shape[1]
Nd=Y_padded_label.shape[1]

In [39]:
Ns

68

In [40]:
Nd

68

In [41]:
class Encoder(torch.nn.Module):

    def __init__(self,src_lang_vocab_size,word_embedding_dim):
        super(Encoder,self).__init__()
        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=src_lang_vocab_size,
                                                       embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,
                                               hidden_size=word_embedding_dim,
                                              batch_first=True)

    def forward(self,X_padded_mini_batch):

        first_embedding_layer_out = self.first_embedding_layer(X_padded_mini_batch)
        encoder_output, (final_encoder_output,final_cell_state) = self.second_lstm_layer(first_embedding_layer_out)

        return encoder_output, (final_encoder_output,final_cell_state)
        
        
        
    

In [42]:
class Decoder(torch.nn.Module):

    def __init__(self,dst_lang_vocab_size,word_embedding_dim):
        super(Decoder,self).__init__()

        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=dst_lang_vocab_size,
                                                       embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,
                                               hidden_size=word_embedding_dim,
                                              batch_first=True)
        self.prediction_layer = torch.nn.Linear(in_features=word_embedding_dim,out_features=dst_lang_vocab_size)
        #self.prediction_layer_activation = torch.nn.Softmax(dim=2)

    def forward(self,Y_padded_input_mini_batch,final_encoder_output,final_cell_state):

        first_embedding_layer_out = self.first_embedding_layer(Y_padded_input_mini_batch)
        decoder_lstm_layer_out, (final_decoder_lstm_layer_out, final_cell_state) = self.second_lstm_layer(first_embedding_layer_out,
                                                                                                         (final_encoder_output,
                                                                                                          final_cell_state))
        prediction = self.prediction_layer(decoder_lstm_layer_out)
        
        return prediction, (final_decoder_lstm_layer_out, final_cell_state)

In [43]:
class Seq2SeqEncDec(torch.nn.Module):

    def __init__(self,src_lang_vocab_size,dst_lang_vocab_size,word_embedding_dim):
        super(Seq2SeqEncDec,self).__init__()

        self.encoder = Encoder(src_lang_vocab_size,word_embedding_dim)
        self.decoder = Decoder(dst_lang_vocab_size,word_embedding_dim)

    def forward(self,X_padded_mini_batch,Y_padded_input_mini_batch):

        encoder_output, (final_encoder_output,final_cell_state) = self.encoder(X_padded_mini_batch)
        y_hat_mini_batch = self.decoder(Y_padded_input_mini_batch,
                                        final_encoder_output,final_cell_state)

        return y_hat_mini_batch
        

In [44]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

cuda


In [45]:
X_padded_train= X_padded[0:13000]
Y_padded_input_train=Y_padded_input[0:13000]
Y_padded_label_train=Y_padded_label[0:13000]

X_padded_test=X_padded[13000:]
Y_padded_input_test=Y_padded_input[13000:]
Y_padded_label_test=Y_padded_label[13000:]

In [ ]:
network = Seq2SeqEncDec(len(Vs),len(Vd),128).to(device)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(network.parameters())
num_epochs = 250
mb_size = 65

for epoch in range(num_epochs):
    for i in range(X_padded_train.shape[0]//mb_size):

        X_train_mb = X_padded_train[i*mb_size:(i+1)*mb_size]
        Y_input_mb = Y_padded_input_train[i*mb_size:(i+1)*mb_size]
        Y_label_mb = Y_padded_label_train[i*mb_size:(i+1)*mb_size]
        Y_label_mb = Y_label_mb.reshape(Y_label_mb.shape[0]*Y_label_mb.shape[1],)
        
        X_train_mb, Y_input_mb, Y_label_mb = X_train_mb.to(device), Y_input_mb.to(device), Y_label_mb.to(device)

        y_hat_train_mb = network(X_train_mb,Y_input_mb)
        y_hat_train_mb = y_hat_train_mb[0]
        y_hat_train_mb = y_hat_train_mb.reshape(y_hat_train_mb.shape[0]*y_hat_train_mb.shape[1],
                                                y_hat_train_mb.shape[2])

        loss_fn_value = loss_fn(y_hat_train_mb,Y_label_mb)

        loss_fn_value.backward()
        torch.nn.utils.clip_grad_norm_(network.parameters(),max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()

        print("Epoch # {}, Time Step # {}, Loss Value = {}".format(epoch,i,loss_fn_value))
        
        


Epoch # 0, Time Step # 0, Loss Value = 8.861103057861328
Epoch # 0, Time Step # 1, Loss Value = 8.836051940917969
Epoch # 0, Time Step # 2, Loss Value = 8.813777923583984
Epoch # 0, Time Step # 3, Loss Value = 8.78075885772705
Epoch # 0, Time Step # 4, Loss Value = 8.765170097351074
Epoch # 0, Time Step # 5, Loss Value = 8.717643737792969
Epoch # 0, Time Step # 6, Loss Value = 8.717636108398438
Epoch # 0, Time Step # 7, Loss Value = 8.658839225769043
Epoch # 0, Time Step # 8, Loss Value = 8.630016326904297
Epoch # 0, Time Step # 9, Loss Value = 8.569209098815918
Epoch # 0, Time Step # 10, Loss Value = 8.496706008911133
Epoch # 0, Time Step # 11, Loss Value = 8.457503318786621
Epoch # 0, Time Step # 12, Loss Value = 8.292996406555176
Epoch # 0, Time Step # 13, Loss Value = 8.309606552124023
Epoch # 0, Time Step # 14, Loss Value = 8.154787063598633
Epoch # 0, Time Step # 15, Loss Value = 8.070028305053711
Epoch # 0, Time Step # 16, Loss Value = 7.864745140075684
Epoch # 0, Time Step # 17

In [47]:
torch.save(network.state_dict(), "model.pth")

In [49]:
def generate_translation(eng_sentence):
    tokenized_eng_sentence=src_sent_tokenizer.tokenize(eng_sentence)
    token_ids=src_sent_tokenizer.convert_tokens_to_ids(tokenized_eng_sentence)
    token_ids_tensor=torch.tensor(token_ids)
    token_ids_tensor=torch.unsqueeze(token_ids_tensor,0)
    
    if torch.cuda.is_available():
        device=torch.device("cuda")
        token_ids_tensor=token_ids_tensor.to(device)
        
    encoder_outputs,(final_encoder_output,final_candidate_cell_state)=encoder(token_ids_tensor)
    decoder_first_time_step_input=torch.tesnor([[0]])

    if torch.cuda.is_available():
        encoder_outputs=encoder_outputs.to(device)
        final_encoder_output=final_encoder_output.to(device)
        final_candidate_cell_state=final_candidate_cell_state.to(device)
        decoder_first_time_step_input=decoder_first_time_step_input.to(device)
        
    decoder_first_time_step_output,(hidden_decoder_output,hidden_decoder_cell_state)=decoder(decoder_first_time_step_input,final_encoder_output,final_candidate_cell_state)
    generated_token_id=torch.argmax
        
    
        